In [ ]:
import pandas as pd
pd.set_option("display.max_seq_items", None)  
pd.set_option("display.width", None)         

In [ ]:
raw_data = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/ret_sample_update.csv")

In [ ]:
data = raw_data.copy()

In [ ]:
# for 2005 to 2025, read pickle file get gvkeys into a dictionary and delete the dataframe to free up memory
gvkeys_dict = {}
for year in range(2005, 2026):
    text_data = pd.read_pickle(f"C:\\Users\\alexi\\OneDrive\\Documents\\école\\McGill-FIAM\\2025\\Hackathon-Final-2025\\DATA ASSET MANAGEMENT HACKATHON 2025 FINALS\\TEXT DATA US by YEAR\\{year}\\text_us_{year}.pkl")
    gvkeys_dict[year] = text_data['gvkey'].unique().tolist()
    del text_data

In [ ]:
# For every gvkey in the dictionary, filter the data dataframe to only include rows with those gvkeys for the corresponding year
filtered_data_list = []
for year in range(2005, 2026):
    # data from that year contains only gvkeys in gvkeys_dict[year]
    filtered_data = data[(data['year'] == year) & (data['gvkey'].isin(gvkeys_dict[year]))]
    filtered_data_list.append(filtered_data)

# concatenate all filtered dataframes
filtered_data = pd.concat(filtered_data_list, ignore_index=True)
del filtered_data_list

In [ ]:
# Keep columns: date, excntry, stock_ret, year, month, char_date, market_equity, be_me, ni_me, at_gr1, tangibility, at_be, debt_me, div12m_me, eqpo_me, eqnetis_at, debt_iss, ni_be, profit_sale, gp_at, turnover_126d
filtered_data = filtered_data[['date', 'gvkey', 'excntry', 'stock_ret', 'year', 'month', 'char_date', 'market_equity', 'be_me', 'ni_me', 'at_gr1', 'tangibility', 'at_be', 'debt_me', 'div12m_me', 'eqpo_me', 'eqnetis_at', 'dbnetis_at', 'ni_be', 'ebit_sale', 'gp_at', 'turnover_126d']]

In [ ]:
joining_table = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/North America Company Name Merge by DataDate-GVKEY-IID.csv")
# rename datadate to date
joining_table = joining_table.rename(columns={"datadate": "date"})

In [ ]:
# --- Keys côté filtered_data ---
# NOTE: char_date dans ret_sample_update est souvent un "trading month-end" (ex: 2021-01-29),
# alors que joining_table peut contenir la fin de mois calendrier (ex: 2021-01-31).
# Pour éviter de perdre des mois entiers au merge, on normalise les deux côtés à la FIN DE MOIS CALENDAIRE.
filtered_data['gvkey_key'] = pd.to_numeric(filtered_data['gvkey'], errors='coerce').astype('Int64')

filtered_data['char_date_key'] = (
    pd.to_datetime(filtered_data['char_date'].astype(str), format='%Y%m%d', errors='coerce')
      .dt.to_period('M')
      .dt.to_timestamp('M')   # month-end calendrier
)

# --- Keys côté joining_table ---
joining_table['gvkey_key'] = pd.to_numeric(joining_table['gvkey'], errors='coerce').astype('Int64')
joining_table['char_date_key'] = (
    pd.to_datetime(joining_table['date'], errors='coerce')
      .dt.to_period('M')
      .dt.to_timestamp('M')   # month-end calendrier
)

# (optionnel) éviter les duplications si plusieurs lignes par gvkey-date
joining_table = joining_table.drop_duplicates(subset=['gvkey_key', 'char_date_key'])

# --- Merge sur gvkey + char_date_key (month-end calendrier) ---
filtered_data = filtered_data.merge(
    joining_table[['gvkey_key', 'char_date_key', 'tic', 'conm']],
    on=['gvkey_key', 'char_date_key'],
    how='left'
)


In [ ]:
sector_mapping = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/Sector Info SIC and GIC codes All Countries to merge by GVKEY and Date.csv")

In [ ]:
# --- Keys côté filtered_data ---
filtered_data['gvkey_key'] = pd.to_numeric(filtered_data['gvkey'], errors='coerce').astype('Int64')
filtered_data['date_key'] = (
    pd.to_datetime(filtered_data['date'].astype(str), format='%Y%m%d', errors='coerce')
      .dt.to_period('M')
      .dt.to_timestamp('M')   # month-end calendrier
)

# --- Keys sector_mapping ---
# Le fichier SIC/GICS contient souvent 'date' (trading month-end) ET parfois 'eom' (calendar month-end).
# On privilégie 'eom' si présent, sinon on retombe sur 'date', puis on normalise en month-end calendrier.
cols = [c for c in ['gvkey', 'date', 'eom', 'gics', 'sic', 'naics'] if c in sector_mapping.columns]
sm = sector_mapping[cols].copy()

sm['gvkey_key'] = pd.to_numeric(sm['gvkey'], errors='coerce').astype('Int64')

date_col = 'eom' if 'eom' in sm.columns else 'date'
# Supporte int YYYYMMDD, string YYYYMMDD, ou YYYY-MM-DD
sm['_raw_date'] = sm[date_col].astype(str).str.replace('-', '').str.slice(0, 8)
sm['date_key'] = (
    pd.to_datetime(sm['_raw_date'], format='%Y%m%d', errors='coerce')
      .dt.to_period('M')
      .dt.to_timestamp('M')
)

sm = sm.drop(columns=['_raw_date'])
# éviter duplications
sm = sm.drop_duplicates(subset=['gvkey_key', 'date_key'])

# --- Merge ---
filtered_data = (
    filtered_data.merge(
        sm[['gvkey_key', 'date_key', 'gics', 'sic', 'naics']],
        on=['gvkey_key', 'date_key'],
        how='left'
    )
    .drop(columns=['gvkey_key', 'date_key'])
)


In [ ]:
# 1) Extraire le code secteur GICS (2 premiers chiffres) à partir du code GICS 8 chiffres
# ex: 20101010 -> 20
filtered_data['gics_sector_code'] = (
    pd.to_numeric(filtered_data['gics'], errors='coerce')
      .floordiv(10**6)
      .astype('Int64')
)

# 2) Mapping MSCI / GICS (11 secteurs)
gics_sector_map = {
    10: "Energy",
    15: "Materials",
    20: "Industrials",
    25: "Consumer Discretionary",
    30: "Consumer Staples",
    35: "Health Care",
    40: "Financials",
    45: "Information Technology",
    50: "Communication Services",
    55: "Utilities",
    60: "Real Estate",
}

# 3) Ajouter le nom du secteur
filtered_data['gics_sector_name'] = filtered_data['gics_sector_code'].map(gics_sector_map)
filtered_data

In [ ]:
filtered_data['tic'].unique()

In [ ]:
from pathlib import Path
import pandas as pd
import ast
import re

# Répertoire (relatif au notebook)
SP500_DIR = Path("sp500-master/sp500_constituants_2005_2024")

def norm_tic(s):
    """Normalise un ticker pour matcher les listes S&P500."""
    if pd.isna(s):
        return pd.NA
    s = str(s).strip().upper()
    # Harmoniser BRK.B vs BRK-B, BF.B vs BF-B, etc. (tes fichiers semblent utiliser '.')
    s = s.replace("-", ".")
    return s

def parse_ticker_list_cell(x):
    """Parse une cellule du type "['A', 'AAPL', ...]" -> list[str]."""
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return [str(t) for t in x]
    s = str(x).strip()
    # Essaye literal_eval
    try:
        v = ast.literal_eval(s)
        if isinstance(v, list):
            return [str(t) for t in v]
    except Exception:
        pass
    # Fallback regex: tout ce qui ressemble à un token alpha-num/./-
    return re.findall(r"[A-Z0-9\.\-]+", s.upper())

# 1) Construire une table (year, tic_norm) à partir des fichiers annuels
members = []
for year in range(2005, 2025):  # tes fichiers sont 2005..2024
    fp = SP500_DIR / f"{year}-sp500-ticker-list.csv"
    if not fp.exists():
        continue
    dfy = pd.read_csv(fp)
    if 'tickers' not in dfy.columns:
        continue

    # On prend la dernière ligne (souvent la liste à la fin d'année), sinon concat toutes les lignes
    # Ici: concat toutes les lignes, puis unique.
    all_tickers = []
    for x in dfy['tickers'].dropna().tolist():
        all_tickers.extend(parse_ticker_list_cell(x))

    all_tickers = [norm_tic(t) for t in all_tickers]
    all_tickers = [t for t in all_tickers if pd.notna(t)]

    if all_tickers:
        members.append(pd.DataFrame({'year': year, 'tic_norm': pd.unique(all_tickers)}))

sp500_members = pd.concat(members, ignore_index=True).dropna().drop_duplicates()

# 2) Filtrer filtered_data par année (S&P500 de l'année correspondante)
fd = filtered_data.copy()
fd['tic_norm'] = fd['tic'].map(norm_tic)

filtered_data = (
    fd.merge(sp500_members, on=['year', 'tic_norm'], how='inner')
      .drop(columns=['tic_norm'])
)


In [ ]:
# Diagnostic: vérifier la couverture mensuelle (utile pour détecter des mois entiers qui disparaissent)
month_counts = (filtered_data.groupby(["year", "month"]).size().reset_index(name="n_rows"))
display(month_counts[month_counts["year"].between(2020, 2021)].sort_values(["year","month"]))


In [ ]:
# Forward fill values par gvkey, trié par date pour les colonnes: stock_ret, 'market_equity', 'be_me', 'ni_me', 'at_gr1', 'tangibility', 'at_be', 'debt_me', 'div12m_me', 'eqpo_me', 'eqnetis_at', 'dbnetis_at', 'ni_be', 'ebit_sale', 'gp_at', 'turnover_126d', 'gics', 'sic', 'naics', 'gics_sector_code', 'gics_sector_name'
filtered_data = filtered_data.sort_values(by=['gvkey', 'date'])
cols_to_ffill = ['stock_ret', 'market_equity', 'be_me', 'ni_me', 'at_gr1', 'tangibility', 'at_be', 'debt_me', 'div12m_me', 'eqpo_me', 'eqnetis_at', 'dbnetis_at', 'ni_be', 'ebit_sale', 'gp_at', 'turnover_126d', 'gics', 'sic', 'naics', 'gics_sector_code', 'gics_sector_name']
filtered_data[cols_to_ffill] = (filtered_data.groupby('gvkey')[cols_to_ffill].ffill())

In [ ]:
# Réduction de la taille des données
# Garder données entre 2021 et 2023 inclus
# Éliminer la moitié des tickers

filtered_data = filtered_data[(filtered_data['year'] >= 2021) & (filtered_data['year'] <= 2023)]
filtered_data = filtered_data.sort_values(by=['gvkey'])
unique_gvkeys = filtered_data['gvkey'].unique()
reduced_gvkeys = unique_gvkeys[::2]  # garder un gvkey sur deux
filtered_data = filtered_data[filtered_data['gvkey'].isin(reduced_gvkeys)]

In [ ]:
# Afficher le nombre de tickers unique par année
for year in range(2005, 2026):
    n_unique_tickers = filtered_data[filtered_data['year'] == year]['gvkey'].nunique()
    print(f"Year {year}: {n_unique_tickers} unique tickers")

In [ ]:
filtered_data.to_csv("yfinance/filtered_sp500_data.csv", index=False)

In [ ]:
filtered_data.head()

In [ ]:
filtered_data['date'].dtypes

In [ ]:
filtered_data.columns